
# Pytorch + HuggingFace 
## KoElectra Model
박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
# !wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
# !wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sentiment_data.tsv', sep='\t')
df.head()

,사람문장1,시스템응답1,사람문장2,시스템응답2,감정_대분류
0,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,기쁨
1,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,불안
2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,슬픔
3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.,서로 의견이 맞지 않아 마음이 불편하시겠어요. 부녀회의 참석에 대해 어떻게 하고 싶...,나뿐만 아니라 앞집도 부녀회의 시간을 맞출 수 없대. 여러 사람과 상의해서 부녀회장...,여러 사람과 의견을 종합하여 부녀회장과 대화를 하시기로 하셨군요.,분노
4,어제 건강검진을 받았는데 상태가 많이 안 좋네.,어디가 편찮으신지 여쭤봐도 될까요?,대장 내시경을 했는데 용종이 나와서 제거했거든. 식단 관리를 좀 해야 할 것 같아.,건강한 음식 위주로 음식관리하시면서 금방 괜찮아지시길 바랄게요. 기운 내세요.,상처


In [22]:
df.columns

Index(['사람문장1', '시스템응답1', '사람문장2', '시스템응답2', '감정_대분류'], dtype='object')

In [23]:
sent_list = df['감정_대분류'].unique()
sent_list

array(['기쁨', '불안', '슬픔', '분노', '상처', '당황'], dtype=object)

In [24]:
sent_to_idx = {}

for i, sent in enumerate(sent_list):
  sent_to_idx[sent] = i
sent_to_idx

{'기쁨': 0, '당황': 5, '분노': 3, '불안': 1, '상처': 4, '슬픔': 2}

In [25]:
for i in range(len(df)):
  df.loc[i, '감정_대분류'] = sent_to_idx[df.loc[i, '감정_대분류']]
df.head()

,사람문장1,시스템응답1,사람문장2,시스템응답2,감정_대분류
0,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,0
1,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,1
2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,2
3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.,서로 의견이 맞지 않아 마음이 불편하시겠어요. 부녀회의 참석에 대해 어떻게 하고 싶...,나뿐만 아니라 앞집도 부녀회의 시간을 맞출 수 없대. 여러 사람과 상의해서 부녀회장...,여러 사람과 의견을 종합하여 부녀회장과 대화를 하시기로 하셨군요.,3
4,어제 건강검진을 받았는데 상태가 많이 안 좋네.,어디가 편찮으신지 여쭤봐도 될까요?,대장 내시경을 했는데 용종이 나와서 제거했거든. 식단 관리를 좀 해야 할 것 같아.,건강한 음식 위주로 음식관리하시면서 금방 괜찮아지시길 바랄게요. 기운 내세요.,4


In [26]:
for i in range(len(df)):
  df.loc[i, 'document'] = '' + str(df.loc[i, '사람문장1']) + str(df.loc[i, '시스템응답1']) + str(df.loc[i, '사람문장2']) + str(df.loc[i, '시스템응답2'])
df.head()

,사람문장1,시스템응답1,사람문장2,시스템응답2,감정_대분류,document
0,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,0,나도 조카가 생겨! 너무 기뻐.정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요...
1,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,1,거래처와의 다음 계약이 무산될까봐 불안해.많이 걱정되시겠어요. 계약이 무산될 것 같...
2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.나이들어 재정적으로 어려워서 걱정되...
3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.,서로 의견이 맞지 않아 마음이 불편하시겠어요. 부녀회의 참석에 대해 어떻게 하고 싶...,나뿐만 아니라 앞집도 부녀회의 시간을 맞출 수 없대. 여러 사람과 상의해서 부녀회장...,여러 사람과 의견을 종합하여 부녀회장과 대화를 하시기로 하셨군요.,3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.서로...
4,어제 건강검진을 받았는데 상태가 많이 안 좋네.,어디가 편찮으신지 여쭤봐도 될까요?,대장 내시경을 했는데 용종이 나와서 제거했거든. 식단 관리를 좀 해야 할 것 같아.,건강한 음식 위주로 음식관리하시면서 금방 괜찮아지시길 바랄게요. 기운 내세요.,4,어제 건강검진을 받았는데 상태가 많이 안 좋네.어디가 편찮으신지 여쭤봐도 될까요?대...


In [27]:
df2 = df[['document', '감정_대분류']]
df2.rename(columns= {'감정_대분류' : 'label'}, inplace=True)
df2.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,document,label
0,나도 조카가 생겨! 너무 기뻐.정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요...,0
1,거래처와의 다음 계약이 무산될까봐 불안해.많이 걱정되시겠어요. 계약이 무산될 것 같...,1
2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.나이들어 재정적으로 어려워서 걱정되...,2
3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.서로...,3
4,어제 건강검진을 받았는데 상태가 많이 안 좋네.어디가 편찮으신지 여쭤봐도 될까요?대...,4


In [28]:
len(df)

25195

In [29]:
df_train = df2[:20000]
df_test = df2[20000:]
display(df_train.head())
display(df_test.head())

,document,label
0,나도 조카가 생겨! 너무 기뻐.정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요...,0
1,거래처와의 다음 계약이 무산될까봐 불안해.많이 걱정되시겠어요. 계약이 무산될 것 같...,1
2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.나이들어 재정적으로 어려워서 걱정되...,2
3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.서로...,3
4,어제 건강검진을 받았는데 상태가 많이 안 좋네.어디가 편찮으신지 여쭤봐도 될까요?대...,4


,document,label
20000,어제 딸아이가 저녁에 와서 나한테 하소연을 하더라고.따님에게 무슨 일이 있었나요?걔...,3
20001,학교폭력을 당하고 있어. 아무도 나를 도와주지 않아서 너무 슬프고 우울해.혹시 고민...,2
20002,아직 은퇴도 안했는데 재정이 탄탄해.재정이 탄탄하시다니 좋으시겠어요.너무 기뻐서 기...,0
20003,능력없는 상사때문에 화가 나.상사와 문제가 있으신가요?뭘 알지도 못하면서 자리만 차...,3
20004,노후준비를 열심히 한다고 했지만 결과물이 별로 좋지 않아 나 자신이 한심해.어느 정...,5


In [30]:
df_train['id'] = df_train.index
df_test['id'] = df_test.index
df_train = df_train[['id', 'document', 'label']]
df_test = df_test[['id', 'document', 'label']]
display(df_train.head())
display(df_test.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,document,label
0,0,나도 조카가 생겨! 너무 기뻐.정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요...,0
1,1,거래처와의 다음 계약이 무산될까봐 불안해.많이 걱정되시겠어요. 계약이 무산될 것 같...,1
2,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.나이들어 재정적으로 어려워서 걱정되...,2
3,3,아파트 부녀회장과 부녀회의 참석 문제로 의견 충돌이 생겼어. 정말 스트레스받아.서로...,3
4,4,어제 건강검진을 받았는데 상태가 많이 안 좋네.어디가 편찮으신지 여쭤봐도 될까요?대...,4


,id,document,label
20000,20000,어제 딸아이가 저녁에 와서 나한테 하소연을 하더라고.따님에게 무슨 일이 있었나요?걔...,3
20001,20001,학교폭력을 당하고 있어. 아무도 나를 도와주지 않아서 너무 슬프고 우울해.혹시 고민...,2
20002,20002,아직 은퇴도 안했는데 재정이 탄탄해.재정이 탄탄하시다니 좋으시겠어요.너무 기뻐서 기...,0
20003,20003,능력없는 상사때문에 화가 나.상사와 문제가 있으신가요?뭘 알지도 못하면서 자리만 차...,3
20004,20004,노후준비를 열심히 한다고 했지만 결과물이 별로 좋지 않아 나 자신이 한심해.어느 정...,5


In [35]:
df_train_0 = df_train[df_train['label'].isin([0, 1])]
df_test_0 = df_test[df_test['label'].isin([0, 1])]

In [17]:
df_train_0.to_csv('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_train_0.tsv', sep='\t', index=False)
df_test_0.to_csv('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_test_0.tsv', sep='\t', index=False)

NameError: ignored

In [107]:
df_train.to_csv('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_train.tsv', sep='\t', index=False)
df_test.to_csv('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_test.tsv', sep='\t', index=False)

In [37]:
cd /content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data

/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data


In [3]:
!head sent_test.tsv

id	document	label
20000	어제 딸아이가 저녁에 와서 나한테 하소연을 하더라고.따님에게 무슨 일이 있었나요?걔가 이번에 새로 직장에 들어갔는데 같이 입사한 동기와 비교당하나 봐. 듣는 내내 정말 속상했어.따님분이 상처받으셨을 것 같아요. 많이 위로해주세요.	3
20001	학교폭력을 당하고 있어. 아무도 나를 도와주지 않아서 너무 슬프고 우울해.혹시 고민을 해결할만한 방법이 있을까요?이렇게 당하고만 있을 순 없으니 선생님과 부모님들께 말씀드려야겠어.선생님과 부모님께 말씀드리는 방법을 택하셨군요.	2
20002	아직 은퇴도 안했는데 재정이 탄탄해.재정이 탄탄하시다니 좋으시겠어요.너무 기뻐서 기분이 좋네.기분이 좋으시니 저도 좋아요.	0
20003	능력없는 상사때문에 화가 나.상사와 문제가 있으신가요?뭘 알지도 못하면서 자리만 차지하고 앉아있으니 보는 것만으로도 구역질 나.상사에 대한 미움이 가득하군요.	3
20004	노후준비를 열심히 한다고 했지만 결과물이 별로 좋지 않아 나 자신이 한심해.어느 정도 이루었다고 생각하시나요?오십 퍼센트 정도는 이루었다고 생각하는데 많이 부족하네.매우 부족하다고 느끼시나 봐요. 앞으로는 어떻게 하실 계획인가요?	5
20005	아내의 출산일이 다가와서 걱정되고 초조해.아내의 출산으로 걱정하고 계시는군요. 출산 준비는 다 마치신 건가요?일정에 맞춰서 산부인과에 입원만 하면 돼. 그때까지 옆에서 잘 도와줘야지.아내가 안심되도록 잘 보살펴주기로 했네요. 배우자가 순산하시길 바랍니다.	1
20006	돈모으는 것에 환멸이 나.무슨 일이 있나요?열심히 모아도 빚갚는데 다써서 모이지가 않아.잘 이겨내실거라 믿어요.	2
20007	건강검진을 받았는데 별문제 없다네.결과가 좋게 나와서 기분이 좋으시군요.앞으로도 관리 잘하면 병 없이 건강한 삶을 살 수 있을 것 같아.앞으로의 건강관리가 중요하군요. 어떻게 관리하고 계세요?	0
20008	결혼을 왜 했을까.왜 그러세요?남편이랑 너무 안맞아.고민이 많으시겠어요.	3


In [ ]:
# !head ratings_train.txt
# !head ratings_test.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [18]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [19]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기 

In [20]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    # 중복제거
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [39]:
train_dataset = NSMCDataset('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_train.tsv')
test_dataset = NSMCDataset('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_test.tsv')

                 id         label
count  19784.000000  19784.000000
mean    9963.017641      2.472503
std     5774.855863      1.680143
min        0.000000      0.000000
25%     4953.750000      1.000000
50%     9941.500000      2.000000
75%    14965.250000      4.000000
max    19999.000000      5.000000
                 id        label
count   5177.000000  5177.000000
mean   22594.429206     2.467645
std     1499.255134     1.670202
min    20000.000000     0.000000
25%    21297.000000     1.000000
50%    22592.000000     3.000000
75%    23892.000000     4.000000
max    25194.000000     5.000000


In [21]:
train_0_dataset = NSMCDataset('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_train_0.tsv')
test_0_dataset = NSMCDataset('/content/drive/MyDrive/Dialog-KoELECTRA/pretrain/setiment_data/sent_train_0.tsv')

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

                 id        label
count   6608.000000  6608.000000
mean    9947.756205     0.503935
std     5749.785393     0.500022
min        0.000000     0.000000
25%     4988.750000     0.000000
50%     9952.000000     1.000000
75%    14904.250000     1.000000
max    19995.000000     1.000000
                 id        label
count   6608.000000  6608.000000
mean    9947.756205     0.503935
std     5749.785393     0.500022
min        0.000000     0.000000
25%     4988.750000     0.000000
50%     9952.000000     1.000000
75%    14904.250000     1.000000
max    19995.000000     1.000000


# Create Model

In [47]:
del model

In [22]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
# model.load_state_dict(torch.load("model.pt"))

FileNotFoundError: ignored

In [23]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# Learn

In [24]:
epochs = 5
batch_size = 16

In [25]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_0_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_0_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [11]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [26]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/413 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 67.85846674442291 Accuracy: tensor(0.6225, device='cuda:0')
Batch Loss: 103.50490036606789 Accuracy: tensor(0.7769, device='cuda:0')
Batch Loss: 126.91659006476402 Accuracy: tensor(0.8277, device='cuda:0')
Batch Loss: 144.09455036744475 Accuracy: tensor(0.8586, device='cuda:0')
Train Loss: 146.08535335212946 Accuracy: tensor(0.8614, device='cuda:0')


  0%|          | 0/413 [00:00<?, ?it/s]

Batch Loss: 13.767267223447561 Accuracy: tensor(0.9606, device='cuda:0')
Batch Loss: 28.521589348092675 Accuracy: tensor(0.9566, device='cuda:0')
Batch Loss: 41.8830837700516 Accuracy: tensor(0.9569, device='cuda:0')
Batch Loss: 54.69941931962967 Accuracy: tensor(0.9572, device='cuda:0')
Train Loss: 56.2323172558099 Accuracy: tensor(0.9572, device='cuda:0')


...

Batch Loss: 9.77637917548418 Accuracy: tensor(0.9694, device='cuda:0')
Batch Loss: 19.525315701961517 Accuracy: tensor(0.9697, device='cuda:0')
Batch Loss: 32.019315887242556 Accuracy: tensor(0.9669, device='cuda:0')
Batch Loss: 40.92671818472445 Accuracy: tensor(0.9683, device='cuda:0')
Train Loss: 41.57750416826457 Accuracy: tensor(0.9688, device='cuda:0')


  0%|          | 0/413 [00:00<?, ?it/s]

Batch Loss: 7.750239523127675 Accuracy: tensor(0.9806, device='cuda:0')
Batch Loss: 15.09118183515966 Accuracy: tensor(0.9784, device='cuda:0')
Batch Loss: 22.05489336606115 Accuracy: tensor(0.9798, device='cuda:0')
Batch Loss: 29.612366429530084 Accuracy: tensor(0.9794, device='cuda:0')
Train Loss: 30.685206430964172 Accuracy: tensor(0.9794, device='cuda:0')


  0%|          | 0/413 [00:00<?, ?it/s]

Batch Loss: 7.12345379544422 Accuracy: tensor(0.9819, device='cuda:0')
Batch Loss: 12.086724788416177 Accuracy: tensor(0.9841, device='cuda:0')
Batch Loss: 16.884516252204776 Accuracy: tensor(0.9848, device='cuda:0')
Batch Loss: 22.10269091743976 Accuracy: tensor(0.9856, device='cuda:0')
Train Loss: 22.8371112793684 Accuracy: tensor(0.9858, device='cuda:0')


In [13]:
F.c

AttributeError: ignored

In [27]:
losses, accuracies

([146.08535335212946,
  56.2323172558099,
  41.57750416826457,
  30.685206430964172,
  22.8371112793684],
 [tensor(0.8614, device='cuda:0'),
  tensor(0.9572, device='cuda:0'),
  tensor(0.9688, device='cuda:0'),
  tensor(0.9794, device='cuda:0'),
  tensor(0.9858, device='cuda:0')])

테스트 데이터셋 정확도 확인하기

In [28]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/413 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy: tensor(0.9924, device='cuda:0')


In [29]:
# 모델 저장하기
torch.save(model.state_dict(), "model_binary_teset.pt")

In [30]:
data = [1, '어제 건강검진을 받으러 갔는데 암이 나왔다네. 난 얼마 못살것 같아']

In [37]:
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  print(predicted)
  # print('예측값: ', y_pred, '실제: ', y_batch, '원문 :', attention_masks_batch)

  0%|          | 0/413 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1], device='cuda:0')
tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], device='cuda:0')
tensor([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1], device='cuda:0')
tensor([0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], device='cuda:0')
tensor([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1], device='cuda:0')
tensor([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0], device='cuda:0')
tensor([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0], device='cuda:0')
tensor([1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')
tensor([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0], device='cuda:0')
tensor([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0], device='cuda:0')
tensor([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0], device='cuda:0')
tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 

In [39]:
next(iter(test_loader))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[tensor([[    2,  7064,  4292,  ...,     0,     0,     0],
         [    2,  7226,  2183,  ...,     0,     0,     0],
         [    2,  7015,  4660,  ...,     0,     0,     0],
         ...,
         [    2,  6652,  4073,  ...,     0,     0,     0],
         [    2,  8258, 14586,  ...,     0,     0,     0],
         [    2,  6294,  4073,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0])]